# YOLOv11 Finetuning - Knife & Scissors Detection

## 1. Setup & Imports

In [ ]:
%pip install ultralytics

In [ ]:
import os
import pandas as pd
from datetime import datetime
from ultralytics import YOLO

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

output_dir = f"results/{timestamp}"
os.makedirs(output_dir, exist_ok=True)

print(f"Output directory: {output_dir}")

## 2. Finetuning
- Model: YOLOv11n (nano - lightweight)
- Epochs: 50 (early stopping patience=10)
- Image size: 640x640
- Augmentation: Mosaic 0.8, Mixup 0.1
- Regularization: Dropout 0.15, Weight decay 0.0005

In [ ]:
model = YOLO("yolo11s.pt")

model.train(
    data="./dataset/merge.yolov11/data.yaml",
    epochs=50,
    patience=10,
    batch=-1,
    imgsz=640,
    project=output_dir,
    name="train",
    
    # === Loss ===
    box=7.5,
    cls=1.5,      # 0.5 → 1.5
    dfl=1.5,
    
    # === Regularization ===
    dropout=0.2,
    weight_decay=0.0005,
    
    # === Augmentation ===
    # Geometric
    fliplr=0.5,
    degrees=15,       # Rotate ±15°
    scale=0.5,        # Zoom 50%-150%
    translate=0.1,
    
    # Mosaic & Mixup
    mosaic=1.0,
    mixup=0.15,
    copy_paste=0.1,
    
    # Color
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    
    # Random erasing
    erasing=0.3,
)

## 3. Evaluation

### 3.1 Load trained model

In [ ]:
finetuned_model = YOLO(f"{output_dir}/train/weights/best.pt")

### 3.2 Validation Set Evaluation

In [ ]:
val_metrics = finetuned_model.val(data="./dataset/merge.yolov11/data.yaml")

val_overall = pd.DataFrame({
    "Metric": ["mAP50", "mAP50-95", "mAP75", "Precision", "Recall"],
    "Value": [
        val_metrics.box.map50,
        val_metrics.box.map,
        val_metrics.box.map75,
        val_metrics.box.mp,
        val_metrics.box.mr
    ]
})

val_per_class = pd.DataFrame({
    "Class": ["knife", "scissors"],
    "mAP50": val_metrics.box.ap50,
    "mAP50-95": val_metrics.box.ap,
    "Precision": val_metrics.box.p,
    "Recall": val_metrics.box.r
})

print("=== VAL OVERALL ===")
print(val_overall.to_string(index=False))
print("\n=== VAL PER-CLASS ===")
print(val_per_class.to_string(index=False))

### 3.3 Test Set Evaluation

In [ ]:
test_metrics = finetuned_model.val(
    data="./dataset/merge.yolov11/data.yaml",
    split="test"
)

test_overall = pd.DataFrame({
    "Metric": ["mAP50", "mAP50-95", "mAP75", "Precision", "Recall"],
    "Value": [
        test_metrics.box.map50,
        test_metrics.box.map,
        test_metrics.box.map75,
        test_metrics.box.mp,
        test_metrics.box.mr
    ]
})

test_per_class = pd.DataFrame({
    "Class": ["knife", "scissors"],
    "mAP50": test_metrics.box.ap50,
    "mAP50-95": test_metrics.box.ap,
    "Precision": test_metrics.box.p,
    "Recall": test_metrics.box.r
})

print("=== TEST OVERALL ===")
print(test_overall.to_string(index=False))
print("\n=== TEST PER-CLASS ===")
print(test_per_class.to_string(index=False))

## 4. Export Results

In [ ]:
val_overall.to_csv(f"{output_dir}/val_overall.csv", index=False)
val_per_class.to_csv(f"{output_dir}/val_per_class.csv", index=False)
test_overall.to_csv(f"{output_dir}/test_overall.csv", index=False)
test_per_class.to_csv(f"{output_dir}/test_per_class.csv", index=False)

print(f"All results saved to: {output_dir}/")
print("\nFiles:")
print("  - val_overall.csv")
print("  - val_per_class.csv")
print("  - test_overall.csv")
print("  - test_per_class.csv")
print("  - train/weights/best.pt")